In [1]:
import pandas as pd
import pandas as pd
from tqdm import tqdm
import pandas as pd
import pymongo
from googleapiclient.discovery import build
import pandas as pd
import seaborn as sns
from dotenv import load_dotenv, dotenv_values
import os
import json
import time
from langchain.document_loaders import YoutubeLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
import anthropic
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from openai import OpenAI
import openai
import os
import together
from uuid import uuid4
from langchain_core.output_parsers import StrOutputParser
import ast
import itertools
import numpy as np
import logging
from typing import Any, Dict, List, Mapping, Optional
from langchain_community.embeddings import CohereEmbeddings
import cohere
from pydantic import Extra, Field, root_validator
from pymongo import MongoClient
import pinecone
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

from IPython.display import display, Markdown

from anthropic import Anthropic
import json
import re
import textwrap
from pprint import pprint
from langchain_anthropic import ChatAnthropic
from tqdm import tqdm
import tiktoken

/home/bilal326/Desktop/rag_project2/rag_project/my_virtual_env/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv('benj_cowen_50_summaries_w_emb.csv')

In [4]:
df['Publish_Date'] = pd.to_datetime(df['Publish_Date'])
df['year'] = df['Publish_Date'].dt.year
df['month'] = df['Publish_Date'].dt.month_name()
df['day'] = df['Publish_Date'].dt.day
df = df.drop('Publish_Date', axis=1)

df['embeddings'] = df['embeddings'].apply(lambda x: np.array(ast.literal_eval(x)))

df.tail()

,Author,title,url,Transcript,Type,Detailed_Summary,tokens,embeddings,id,year,month,day
46,Benjamin Cowen,Bitcoin: Bull Market Support Band,https://www.youtube.com/watch?v=TQvaZjdkT8Q,Video Transcript of youtube video 'Bitcoin: Bu...,Youtube video,The video provides a detailed technical analys...,271,"[0.00680542, -0.005558014, -0.010070801, 0.014...",13bf472b-5480-4372-8ffa-fba275e10805,2023,December,17
47,Benjamin Cowen,Bitcoin: Running Return on Investment,https://www.youtube.com/watch?v=2bDez8iqqa4,Video Transcript of youtube video 'Bitcoin: Ru...,Youtube video,The video provides a detailed technical analys...,223,"[0.0069007874, -0.014877319, -0.004627228, 0.0...",9692e141-e5b9-421f-a153-d4f6c110facf,2023,December,15
48,Benjamin Cowen,Bitcoin Dominance,https://www.youtube.com/watch?v=1zR6R6DC_Zo,Video Transcript of youtube video 'Bitcoin Dom...,Youtube video,The video discusses the importance of Bitcoin ...,342,"[0.015235901, -0.034698486, 0.013259888, 0.004...",8fc19b58-0627-407c-905a-e47097a8bed6,2023,June,27
49,Benjamin Cowen,S&amp;P 500 ROI After Yield Curve Inversion,https://www.youtube.com/watch?v=0ZljXuhkPSw,Video Transcript of youtube video 'S&amp;P 500...,Youtube video,The video discusses the historical relationshi...,152,"[0.04336548, -0.004005432, 0.04522705, 0.03826...",9114ff56-55ac-478c-87ee-3268b357c54c,2023,June,21
50,Benjamin Cowen,Where Is The Crypto Bottom? (Worst Case Scenario),https://www.youtube.com/watch?v=jvGjWARiQZg,Video Transcript of youtube video 'Where Is Th...,Youtube video,The video presents a detailed analysis compari...,368,"[0.021453857, 0.01828003, 0.029937744, 0.00276...",957795a4-d587-4a81-984c-4fc02b31dd37,2023,March,10


In [5]:
df_dict_list2 = []

for index, row in df.iterrows():
    data_dict = {
        "id": row['id'],
        "values": row['embeddings'],
        "metadata": {
            "text": row['Detailed_Summary'],
            "Author": row['Author'],
            "title": row['title'],
            "url": row['url'],
            "Type": row['Type'],
            "year": row['year'],
            "month": row['month'],
            "day": row['day']
        }
    }
    df_dict_list2.append(data_dict)

with open("benj_cowen_50_summaries_w_emb2.jsonl", "w") as f:
    for item in df_dict_list2:
        # Convert NumPy arrays to lists
        for key, value in item.items():
            if isinstance(value, np.ndarray):
                item[key] = value.tolist()

        # Serialize the dictionary to JSON and write to file
        json.dump(item, f)
        f.write("\n")


In [6]:
from pinecone import ServerlessSpec, PodSpec, Pinecone
spec = ServerlessSpec(cloud='aws', region='us-west-2')
api_key = "805b93a4-7e60-47df-9184-537be20493ce"
# configuring client
pc = Pinecone(api_key=api_key)

pc.list_indexes()

{'indexes': [{'dimension': 1024,
              'host': 'benjamin-cowen-summ2-x3amlsu.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'benjamin-cowen-summ2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1024,
              'host': 'masterindex1-x3amlsu.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'masterindex1',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimension': 1024,
              'host': 'masterindex2-x3amlsu.svc.apw5-4e34-81fa.pinecone.io',
              'metric': 'cosine',
              'name': 'masterindex2',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-west-2'}},
              'status': {'ready': True, 'state': 'Ready'}},
             {'dimens

In [7]:
import time

index_name = 'masterindex2'

dimensions = 1024
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name= index_name,
        dimension = dimensions,
        metric = "cosine",
        spec=spec
        )

# wait for index to be ready before connecting - cell not finish
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

In [8]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2817}},
 'total_vector_count': 2817}

In [9]:
vector_dim = 1024


def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))


with pc.Index(index_name) as index:
    # Send requests in parallel
    async_results = [
        index.upsert(vectors=ids_vectors_chunk, async_req=True)
        for ids_vectors_chunk in chunks(df_dict_list2, batch_size=100)
    ]
    [async_result.get() for async_result in async_results]

In [10]:
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 2868}},
 'total_vector_count': 2868}